In [80]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [81]:
# For optimization using pygmo
import pygmo as pg
from udp_initial_condition import udp_initial_condition

# For Plotting
import matplotlib.pyplot as plt
from math import pi

# Setup problem hyperparameters:
body_density = 533           # https://sci.esa.int/web/rosetta/-/14615-comet-67p
body_mu = 665.666            # Gravitational parameter for 67P/C-G
target_altitude = 5000**2    # Target altitude squared
final_time = 20*3600.0       # Final time
start_time = 0               # Starting time
time_step = 600.0            # Time step size for trajectory propagation.

# Defining the parameter space for the optimization
#   (Parameters are defined in osculating orbital elements)
a = [5000, 15000] # Semi-major axis
e = [0, 1]        # Eccentricity [0, 1]
o = [0, 2*pi]     # Right ascension of ascending node [0,2*pi]
w = [0, 2*pi]     # Argument of periapsis [0, 2*pi]
i = [0, pi]       # Inclination [0, pi] 
ea = [0, 2*pi]    # Eccentric anomaly [0, 2*pi]

lower_bounds = [a[0], e[0], i[0], o[0], w[0], ea[0]]
upper_bounds = [a[1], e[1], i[1], o[1], w[1], ea[1]]


# Optimization algorithm hyperparameters
population_size = 10 
number_of_generations = 32 




# Setup class
udp = udp_initial_condition(body_density, body_mu, target_altitude, final_time, start_time, time_step, lower_bounds, upper_bounds)

# Setup user defined problem (udp)
prob = pg.problem(udp)

# Setup optimization algorithm
algo = pg.algorithm(pg.sade(gen = number_of_generations))
pop = pg.population(prob = prob, size = population_size)

# Run optimization
algo.set_verbosity(1)
pop = algo.evolve(pop)

# Logs for output
print("Champion fitness value: ", pop.champion_f) 
print("Champion chromosome: ", pop.champion_x) 

# Retrieving champion chromosome
x = pop.champion_x

# Compute optimized trajectory
r_store, v_store, a_store = udp.compute_trajectory(x)

# Plot optimized trajectory
udp.plot_trajectory(r_store)
plt.plot(r_store[1],r_store[2]) 
